In [84]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from scipy.stats import ttest_ind

From the EDA we know,the dataset contains null, duplicate values and some amount of outliers. \
From the EDA we know, that some categorical type data columns are highly imbalanced. \
We have also observed that how distribution differs with the different categories and the different %Churn for different columns.

In [29]:
df=pd.read_csv('data.csv')

In [30]:
df.duplicated().sum()

0

In [31]:
df.isnull().sum()

customerID          0
gender              1
SeniorCitizen       4
Partner             3
Dependents          4
tenure              2
PhoneService        3
MultipleLines       3
InternetService     3
OnlineSecurity      2
OnlineBackup        1
DeviceProtection    2
TechSupport         2
StreamingTV         5
StreamingMovies     0
Contract            2
PaperlessBilling    1
PaymentMethod       4
MonthlyCharges      0
TotalCharges        6
Churn               2
dtype: int64

In [32]:
df.drop(columns=['customerID'],inplace=True)

In [33]:
df.dtypes

gender               object
SeniorCitizen       float64
Partner              object
Dependents           object
tenure              float64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [34]:
df['SeniorCitizen']=df['SeniorCitizen'].astype('category') 

In [51]:
#Cateforical Imputer
cat_impute=SimpleImputer(strategy='most_frequent')
cat_col=df.select_dtypes(include=['category','object']).columns.values
df[cat_col]=cat_impute.fit_transform(df[cat_col])

In [59]:
df[cat_col].isna().sum().sum()

0

In [55]:
#Numerical values, since the number null in numerical columns  are very low we'll just drop them
df.dropna(inplace=True) 

In [58]:
df.isnull().sum().sum()

0

### Hypothesis Testing
We can statistically check whether all the independent variables has significant effect on target variables

In [76]:
numerical_cols = df.select_dtypes(include=[np.number])

# Group by 'Churn' and calculate the mean of numerical columns
class_summary = df.groupby('Churn')[numerical_cols.columns].agg(np.mean) 
class_summary

,tenure,MonthlyCharges,TotalCharges
Churn,,,
No,37.656867,61.523704,2564.930667
Yes,18.137265,74.675496,1545.221367


We can see the difference between in mean of all continious values. \
It suggests that there may be some relation between the continious values and Churn.

H0(Null Hypothesis):No Significant difference between the independent feature with different category of the target variable

H1(Alternate Hypothesis):There is significant difference between features with different categories of Target Variable.

In [88]:
for i in df.select_dtypes(include='number').columns.values:
    x=np.array(df[df.Churn=='No'][i])
    y=np.array(df[df.Churn=='Yes'][i])
    t,p_value=ttest_ind(x,y,axis=0,equal_var=False)
    print('{} P_value:{}'.format('\033[1m',p_value))
    if p_value<0.05:
        print('{} Reject Null Hypothesis.{} of Churned and non-churned Customer are not the same'.format('\033[1m',i))
    else:
        print('{} Fail to Reject Null Hypothesis.{} of Churned and non-churned Customer are  same'.format('\033[1m',i))
    print('\n')

 P_value:2.7539972292832127e-221
 Reject Null Hypothesis.tenure of Churned and non-churned Customer are not the same


 P_value:7.088838995641746e-69
 Reject Null Hypothesis.MonthlyCharges of Churned and non-churned Customer are not the same


 P_value:5.207968082025042e-73
 Reject Null Hypothesis.TotalCharges of Churned and non-churned Customer are not the same




In [82]:
df.select_dtypes(include='number').columns.values

array(['tenure', 'MonthlyCharges', 'TotalCharges'], dtype=object)